In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('../data/Merged/train.csv')
train

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
...,...,...
2271,"Recuperamos la historia de Aleixandra, la jove...",0
2272,"Reproches, tensión y sinceridad: la comida en ...",0
2273,"RT @ElMundoOpinion: ""PSOE, PP, Ciudadanos y Vo...",0
2274,Rusia cita al embajador español por unas decla...,0


In [3]:
test = pd.read_csv('../data/Merged/test.csv')
test

,text,label
0,MAESTRA DE *NUMBER* AÑOS QUE TUVO RELACIONES C...,1
1,Oxford lanza sus propios exámenes de certifica...,0
2,La RAE estudia incluir «machirulo» en el Dicci...,0
3,Malala Yousafzai anuncia que estudiará en Oxfo...,0
4,Nombran a Ricardo Arjona nuevo miembro de la R...,1
...,...,...
290,Meryl Streep disfrutó unos premios Oscar tan m...,0
291,EL PLAGIO DE LANA DEL REY A RADIOHEAD FUE ACOR...,1
292,Ricardo Arjona lanza una serie documental por ...,0
293,Raúl Araiza sorprende a Andrea Legarreta con b...,0


In [4]:
def max_length_text(df):
    max_length = 0
    for i in range(df.shape[0]):
        length = np.size(word_tokenize(df.at[i, 'text']))
        if length > max_length: max_length = length
    return max_length

In [16]:
def text_normalization(data):
    data['text'] = data['text'].apply(lambda x: x.lower())
    data['text'] = data['text'].apply((lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x)))

In [17]:
def remove_stop_words(data, language):
    stopwords = nltk.corpus.stopwords.words(language)
    for i in range(data.shape[0]):
        data.at[i, 'text'] = [word for word in nltk.word_tokenize(data.at[i, 'text']) if word not in stopwords]
        data.at[i, 'text'] = ' '.join(data.at[i, 'text'])

In [18]:
def apply_stemming(data, language):
    stemmer = SnowballStemmer(language)
    for i in range(data.shape[0]):
         data.at[i, 'text'] = (' '.join([stemmer.stem(word) for word in data.at[i, 'text'].split()]))

get_matrix representation

In [23]:
def get_matrix(data, representation, vocabulary_length = 2000, stop_words_flag = True, language = 'spanish'):

    df = data.copy(deep = True)
    
    text_normalization(df) # Text normalization
    
    # Stop_words
    if stop_words_flag:
        remove_stop_words(df, language)
    
    apply_stemming(df, language) # Stemming
    
    # Word representation
    if representation == 'BoW':
        count_vectorizer = CountVectorizer(max_features = vocabulary_length)
        matrix = count_vectorizer.fit_transform(df.text)
        
    elif representation == 'tf-idf':
        tfidf_vectorizer = TfidfVectorizer(max_features = vocabulary_length, use_idf=True)
        matrix = tfidf_vectorizer.fit_transform(df.text)
    
    return matrix, df

In [24]:
print(test.at[0, 'text'])

MAESTRA DE *NUMBER* AÑOS QUE TUVO RELACIONES CON UN ALUMNO DE *NUMBER* DARÁ EDUCACIÓN SEXUAL EN EL CONALEP
Estados Unidos.- Luego de que se revelará el caso de una maestra estadounidense de *NUMBER* años de nombre Alexandria Vera que fue acusada de abusar de uno de sus alumno de *NUMBER* años (con quien tuvo un noviazgo e incluso quedó embarazada), el juez que analizó la situación finalmente determinó que la profesora no será encarcelada pero si deberá pagar una multa de USD*NUMBER* mil dólares, alejarse del adolescente para siempre, y dar un servicio comunitario de *NUMBER* horas como profesora en un CONALEP de la Ciudad de México.
El juez señala que el chico involucrado confesó que "tuvo relaciones con su consentimiento" y además sus papás aprobaban el noviazgo:
"Es un caso que llamó la atención de los medios internacionales porque ella tiene *NUMBER* y es muy atractiva, y él es un joven de apenas *NUMBER* años.
Como el juez que iba a determinar el castigo sentía la presión de muchas

In [26]:
matrix, df = get_matrix(test, representation = 'BoW')
print(df.at[0, 'text'])
print(matrix.shape)
print(matrix.toarray())

maestr numb años relacion alumn numb dar educ sexual conalep unid lueg revel cas maestr estadounidens numb años nombr alexandri ver acus abus alumn numb años noviazg inclus qued embaraz juez analiz situacion final determin profesor encarcel si deb pag mult usdnumb mil dolar alej adolescent siempr dar servici comunitari numb hor profesor conalep ciud mexic juez señal chic involucr confes relacion consent ademas papas aprob noviazg cas llam atencion medi internacional numb atract jov apen numb años juez iba determin castig sent presion much person quer met carcel maestr embarg ser objet propi jov padr pid dej libert ademas verd record etap secundari compañer fantas maestr ris asi pens ¿por ser tan sever si algun vez fantas chic decid alexandri pis carcel castig ir conalep dar clas sab fam con asi ret fuert conten si vuelv comet mism delit vez salv pis carcel confes magistr señal tom decision mand prep mexican pon prueb fueg profesor dej clar gust chavit con chic dispuest darl entrad guap

In [27]:
print(matrix[0])

  (0, 1128)	4
  (0, 1292)	7
  (0, 205)	4
  (0, 1623)	2
  (0, 94)	3
  (0, 518)	3
  (0, 655)	1
  (0, 1739)	1
  (0, 1906)	1
  (0, 1117)	1
  (0, 1654)	1
  (0, 304)	2
  (0, 747)	1
  (0, 1281)	1
  (0, 1942)	1
  (0, 34)	1
  (0, 983)	1
  (0, 1563)	1
  (0, 676)	1
  (0, 1059)	3
  (0, 110)	1
  (0, 1771)	2
  (0, 830)	1
  (0, 582)	2
  (0, 1516)	3
  :	:
  (0, 351)	1
  (0, 915)	1
  (0, 621)	1
  (0, 519)	1
  (0, 710)	1
  (0, 477)	1
  (0, 549)	1
  (0, 657)	1
  (0, 266)	1
  (0, 48)	1
  (0, 921)	1
  (0, 120)	1
  (0, 985)	1
  (0, 717)	1
  (0, 1188)	1
  (0, 1118)	1
  (0, 502)	1
  (0, 81)	1
  (0, 98)	1
  (0, 501)	1
  (0, 1718)	1
  (0, 1409)	1
  (0, 1353)	1
  (0, 1775)	1
  (0, 1559)	1


In [28]:
df

,text,label
0,maestr numb años relacion alumn numb dar educ ...,1
1,oxford lanz propi examen certif nivel ingles e...,0
2,rae estudi inclu « machirul » diccionari acade...,0
3,malal yousafzai anunci estudi oxford nobel paz...,0
4,nombr ricard arjon nuev miembr real academi es...,1
...,...,...
290,meryl streep disfrut premi oscar tan mexican ¡...,0
291,plagi lan rey radioh acord ambos lan rey anunc...,1
292,ricard arjon lanz seri documental youtub sol c...,0
293,raul araiz sorprend andre legarret bes boc vay...,0


In [29]:
matrix, df = get_matrix(test, representation = 'tf-idf')
print(df.at[0, 'text'])
print(matrix.shape)
print(matrix.toarray())

maestr numb años relacion alumn numb dar educ sexual conalep unid lueg revel cas maestr estadounidens numb años nombr alexandri ver acus abus alumn numb años noviazg inclus qued embaraz juez analiz situacion final determin profesor encarcel si deb pag mult usdnumb mil dolar alej adolescent siempr dar servici comunitari numb hor profesor conalep ciud mexic juez señal chic involucr confes relacion consent ademas papas aprob noviazg cas llam atencion medi internacional numb atract jov apen numb años juez iba determin castig sent presion much person quer met carcel maestr embarg ser objet propi jov padr pid dej libert ademas verd record etap secundari compañer fantas maestr ris asi pens ¿por ser tan sever si algun vez fantas chic decid alexandri pis carcel castig ir conalep dar clas sab fam con asi ret fuert conten si vuelv comet mism delit vez salv pis carcel confes magistr señal tom decision mand prep mexican pon prueb fueg profesor dej clar gust chavit con chic dispuest darl entrad guap

In [30]:
print(matrix[0])

  (0, 1559)	0.04986403033763974
  (0, 1775)	0.029450468850242727
  (0, 1353)	0.03010380708550428
  (0, 1409)	0.03725603390799652
  (0, 1718)	0.04337679317119601
  (0, 501)	0.052429212634438345
  (0, 98)	0.06701612076924612
  (0, 81)	0.07014909311175736
  (0, 502)	0.06440798794871706
  (0, 1118)	0.04246706952091257
  (0, 1188)	0.06116621721797506
  (0, 717)	0.052997002680150986
  (0, 985)	0.06116621721797506
  (0, 120)	0.059327328922508596
  (0, 921)	0.02798572301563638
  (0, 48)	0.07014909311175736
  (0, 266)	0.07198798140722382
  (0, 657)	0.052997002680150986
  (0, 549)	0.05619435657999736
  (0, 477)	0.03683967575296423
  (0, 710)	0.06565765516486621
  (0, 519)	0.054835890975617446
  (0, 621)	0.05848320413146177
  (0, 915)	0.05768238728920992
  (0, 351)	0.04986403033763974
  :	:
  (0, 1516)	0.18349865165392518
  (0, 582)	0.12434765047220216
  (0, 830)	0.03976856014972311
  (0, 1771)	0.10717244751893658
  (0, 110)	0.05848320413146177
  (0, 1059)	0.2104472793352721
  (0, 676)	0.07014909